# Data processing with Pandas, part 2

This week we will continue developing our skills using [pandas](https://pandas.pydata.org/) to process real data. 

Case: April 2019 happened to be the second warmest April on record globally. 
In this lesson, we will use our data manipulation and analysis skills to analyze weather data from Finland, and see if April 2019 was exceptionally warm also here.

We will cover a number of useful techniques in pandas including:

- renaming columns
- iterating data frame rows and applying functions
- data aggregation
- repeating the analysis task for several input files 


## Input data
In the lesson this week we are using weather observation data from Finland downloaded from NOAA. We have data for 15 different weather obsercation stations from Finland. 


**METADATA:**

- List of available stations: [metadata/6367598020644stn.txt](metadata/6367598020644stn.txt)
- More details about weather observatios per station: [metadata/6367598020644inv.txt](metadata/6367598020644inv.txt)
- Data description (column names): [metadata/3505doc.txt](metadata/3505doc.txt)


The input data for this week is separated with varying amount of spaces (fixed width). The first lines and columns of the data looks like following:

``` 
    
      USAF  WBAN YR--MODAHRMN DIR SPD GUS CLG SKC L M H  VSB MW MW MW MW AW AW AW AW W TEMP DEWP    SLP  ... SD
    028360 99999 191701010600 360   5 *** *** SCT * * *  0.0 ** ** ** ** ** ** ** ** *    3 ****  986.0  ... ** 
    028360 99999 191701011300 ***   0 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *  -14 ****  986.6  ... ** 
    028360 99999 191701012000 ***   0 *** *** BKN * * *  0.0 ** ** ** ** ** ** ** ** *   -7 ****  986.6  ... ** 
    028360 99999 191701020600 360   2 *** *** OVC * * *  0.0 ** ** ** ** ** ** ** ** *  -14 ****  984.7  ... ** 

    
```

**We will develop our analysis workflow using data for one station. Then' we will repeat the same process for all the stations.**

## Reading the data
In order to get startet, let's import pandas: 

In [1]:
import pandas as pd

At this point, you should have already had a look at the input file and how it is structured. We can notice at least two things we need to consider when reading in the data:


<div class="alert alert-info">

**NOTE: Input data structure**

- **Delimiter:** The data are **separated with varying amount of spaces**. If you check out the documentation for the [read_csv() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html), you can see that there are two different ways of doing this. We can either use the `sep` or `delim_whitespace` parameter;  `sep='\s+'` or `delim_whitespace=True` but not both. In this case, we prefer to use `delim_whitespace`.

- **No Data values:** No data values in the NOAA data are coded with varyingg number of `*`. We can tell pandas to consider those characters as NaNs by specifying `na_values=['*', '**', '***', '****', '*****', '******']`.
</div>




In [2]:
fp = r"C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\029350.txt"

# Read data using varying amount of spaces as separator and specifying * characters as NoData values
data = pd.read_csv(fp, delim_whitespace=True, na_values=['*', '**', '***', '****', '*****', '******'])

C:\HYapp\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


- Let's see how the data looks by printing the first five rows with `head()` function

In [3]:
print(data.head())

    USAF   WBAN  YR--MODAHRMN    DIR  SPD  GUS  CLG  SKC   L   M ...    SLP  \
0  29350  99999  191701010600  320.0  5.0  NaN  NaN  CLR NaN NaN ...  993.2   
1  29350  99999  191701011300  320.0  7.0  NaN  NaN  CLR NaN NaN ...  995.1   
2  29350  99999  191701012000  320.0  5.0  NaN  NaN  SCT NaN NaN ...  996.8   
3  29350  99999  191701021300    NaN  0.0  NaN  NaN  OVC NaN NaN ...  996.7   
4  29350  99999  191701022000    NaN  0.0  NaN  NaN  CLR NaN NaN ...  997.0   

   ALT  STP  MAX  MIN  PCP01  PCP06  PCP24  PCPXX  SD  
0  NaN  NaN  NaN  NaN    NaN    NaN    NaN    NaN NaN  
1  NaN  NaN  NaN  NaN    NaN    NaN    NaN    NaN NaN  
2  NaN  NaN  NaN  NaN    NaN    NaN    NaN    NaN NaN  
3  NaN  NaN  NaN  NaN    NaN    NaN    NaN    NaN NaN  
4  NaN  NaN  NaN  NaN    NaN    NaN    NaN    NaN NaN  

[5 rows x 33 columns]


All seems ok. However, we won't be needing all of the 33 columns for detecting warm temperatures in April. We can check all column names by running `data.columns`:

In [4]:
data.columns

Index(['USAF', 'WBAN', 'YR--MODAHRMN', 'DIR', 'SPD', 'GUS', 'CLG', 'SKC', 'L',
       'M', 'H', 'VSB', 'MW', 'MW.1', 'MW.2', 'MW.3', 'AW', 'AW.1', 'AW.2',
       'AW.3', 'W', 'TEMP', 'DEWP', 'SLP', 'ALT', 'STP', 'MAX', 'MIN', 'PCP01',
       'PCP06', 'PCP24', 'PCPXX', 'SD'],
      dtype='object')

Description for all these columns is available in the metadata file [metadata/3505doc.txt](metadata/3505doc.txt). 

**Let's read in the data one more time.** This time, we will read in only some of the columns using the `usecols` parameter. Let's read in columns that might be somehow useful to our analysis, or at least that contain some values that are meaningful to us, including the station name, timestamp, and data about wind and temperature: `'USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'`

In [5]:
data = pd.read_csv(fp, delim_whitespace=True, usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], na_values=['*', '**', '***', '****', '*****', '******'])
data.head()

USAF  YR--MODAHRMN    DIR  SPD  GUS  TEMP  MAX  MIN
0  29350  191701010600  320.0  5.0  NaN   6.0  NaN  NaN
1  29350  191701011300  320.0  7.0  NaN   2.0  NaN  NaN
2  29350  191701012000  320.0  5.0  NaN  -8.0  NaN  NaN
3  29350  191701021300    NaN  0.0  NaN  -4.0  NaN  NaN
4  29350  191701022000    NaN  0.0  NaN -12.0  NaN  NaN

Okay so we can see that the data was successfully read to the DataFrame and we also seemed to be able to convert the asterix (\*) characters into `NaN` -values. 

## Renaming columns

Check again the column names in our DataFrame:

In [6]:
data.columns

Index(['USAF', 'YR--MODAHRMN', 'DIR', 'SPD', 'GUS', 'TEMP', 'MAX', 'MIN'], dtype='object')

The column names that we have are somewhat ackward. Let's change them into more intuitive ones. 
This can be done easily using the `rename()` -method and a dictionary that lists old and new column names.

<div class="alert alert-info">

**Dictionaries**

[Dictionary](https://docs.python.org/2/tutorial/datastructures.html#dictionaries) is a spesific data structure in Python for storing key-value pairs. During this course, we will use dictionaries mainly when renaming columns in a pandas series, but dictionaries are useful for many different purposes! For more information about Python dictionaries, check out [this tutorial](https://realpython.com/python-dicts/).
    
</div>



We can define the new column names using a [dictionary](https://www.tutorialspoint.com/python/python_dictionary.htm) where we determine "`key: value`" -pairs, in which the original column name (the one which will be replaced) is the key, and the new column name is the value.

- Let's change:
   
   - `YR--MODAHRMN` column into `TIME`, 
   - `SPD` into `SPEED`, and
   - `GUS` into `GUST`

In [7]:
# Create the dictionary with old and new names
new_names = {'YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST'}

# Let's see what they look like and what is the type
print(new_names)
print(type(new_names))

{'SPD': 'SPEED', 'YR--MODAHRMN': 'TIME', 'GUS': 'GUST'}
<class 'dict'>


From above we can see that we have successfully created a dictionary that is of type `dict`. 

- Now we can change the column names by passing that dictionary into parameter `columns` in `rename()` -function.

In [8]:
# Rename the columns
data = data.rename(columns=new_names)

# Print the new columns
print(data.columns)

Index(['USAF', 'TIME', 'DIR', 'SPEED', 'GUST', 'TEMP', 'MAX', 'MIN'], dtype='object')


Perfect, now our column names are more easy to understand and use. 

<div class="alert alert-info">

**TASK: Renaming columns**

The temperature values are again in Fahrenheit. As you might guess, we will soon convert these temperatures in to Celsius. In order to avoid confusion with the columns, rename column `TEMP` into `TEMP_F`. Also, we could rename `USAF` as`STATION_NUMBER`.
</div>


In [9]:
# Create the dictionary with old and new names
new_names = {'USAF':'STATION_NUMBER', 'TEMP': 'TEMP_F'}

# Rename the columns
data = data.rename(columns=new_names)

# Check the output
data.head()

STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN
0           29350  191701010600  320.0    5.0   NaN     6.0  NaN  NaN
1           29350  191701011300  320.0    7.0   NaN     2.0  NaN  NaN
2           29350  191701012000  320.0    5.0   NaN    -8.0  NaN  NaN
3           29350  191701021300    NaN    0.0   NaN    -4.0  NaN  NaN
4           29350  191701022000    NaN    0.0   NaN   -12.0  NaN  NaN

## Check dataframe properties

As we learned last week, it's always a good idea to check basic properties of the input data before proceeding with data analysis. Let's check:

- How many rows and columns we have:

In [10]:
data.shape

(559667, 8)

- Top and bottom rows: 

In [11]:
data.head()

STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN
0           29350  191701010600  320.0    5.0   NaN     6.0  NaN  NaN
1           29350  191701011300  320.0    7.0   NaN     2.0  NaN  NaN
2           29350  191701012000  320.0    5.0   NaN    -8.0  NaN  NaN
3           29350  191701021300    NaN    0.0   NaN    -4.0  NaN  NaN
4           29350  191701022000    NaN    0.0   NaN   -12.0  NaN  NaN

In [12]:
data.tail()

STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN
559662           29350  201910012220  990.0    5.0   NaN    37.0  NaN  NaN
559663           29350  201910012250  990.0    7.0   NaN    37.0  NaN  NaN
559664           29350  201910012300  260.0    4.0   NaN    38.0  NaN  NaN
559665           29350  201910012320  990.0    7.0   NaN    37.0  NaN  NaN
559666           29350  201910012350  990.0    5.0   NaN    37.0  NaN  NaN

- Data types of the columns: 

In [13]:
data.dtypes

STATION_NUMBER      int64
TIME                int64
DIR               float64
SPEED             float64
GUST              float64
TEMP_F            float64
MAX               float64
MIN               float64
dtype: object

- Descriptive statistics:

In [14]:
print(data.describe())

       STATION_NUMBER          TIME            DIR          SPEED  \
count        559667.0  5.596670e+05  519843.000000  553514.000000   
mean          29350.0  2.002912e+11     282.237183       6.235060   
std               0.0  1.800321e+09     247.127478       4.216836   
min           29350.0  1.917010e+11       1.000000       0.000000   
25%           29350.0  2.001122e+11     140.000000       3.000000   
50%           29350.0  2.008061e+11     220.000000       6.000000   
75%           29350.0  2.013110e+11     320.000000       9.000000   
max           29350.0  2.019100e+11     990.000000      66.000000   

              GUST         TEMP_F           MAX           MIN  
count  9916.000000  555794.000000  30271.000000  30268.000000  
mean     21.325534      39.039754     43.731030     32.457975  
std       7.367612      18.833681     19.378822     18.427926  
min      11.000000     -37.000000    -28.000000    -48.000000  
25%      15.000000      28.000000     30.000000     23.000

Here we can see that there are varying number of observations per column (see the `count` -information), because some of the columns have missing values.

## Using your own functions in pandas 

Now it's again time to convert temperatures from Fahrenheit to Celsius! Yes, we have already done this many times before, but this time we will learn how to apply self-made functions to data in a pandas DataFrame.
**In short, our task is to define a function for the temperature conversion, and to apply this function for each Celsius value on each row of the DataFrame. Output celsius values should be stored in a new column called** `TEMP_C`.

Knowing how to use your own function in pandas can be really useful when doing your own analyses. Here, we will introduce two different approaches for using function in pandas. First, we will see how we can apply the function row-by-row using a `for`-loop and the `DataFrame.iterrows()`-method, and then we will learn how to apply the method to all rows at once using [DataFrame.apply](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html).

For both of these approaches, we first need to define our temperature conversion function from Fahrenheit to Celsius:

In [15]:
def fahr_to_celsius(temp_fahrenheit):
    """
    Function to convert Fahrenheit temperature into Celsius.

    Parameters
    ----------

    temp_fahrenheit: int | float
        Input temperature in Fahrenheit (should be a number)
        
    Returns
    -------
    
    Temperature in Celsius (float)
    """

    # Convert the Fahrenheit into Celsius and return it
    converted_temp = (temp_fahrenheit - 32) / 1.8
    return converted_temp

**Note:** with such a simple example, we could use the function direcly on a column in the DataFrame in order to conver the values:

In [16]:
data["TEMP_C"] = fahr_to_celsius(data["TEMP_F"])
data.head()

STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29350  191701010600  320.0    5.0   NaN     6.0  NaN  NaN   
1           29350  191701011300  320.0    7.0   NaN     2.0  NaN  NaN   
2           29350  191701012000  320.0    5.0   NaN    -8.0  NaN  NaN   
3           29350  191701021300    NaN    0.0   NaN    -4.0  NaN  NaN   
4           29350  191701022000    NaN    0.0   NaN   -12.0  NaN  NaN   

      TEMP_C  
0 -14.444444  
1 -16.666667  
2 -22.222222  
3 -20.000000  
4 -24.444444

If we want to do something more complicated, we need to know how to apply the function row-by-row.

### Iterating over rows

We can iterate over the rows of Pandas DataFrame by using the `iterrows()` -method and use the function one row at a time.


When iterating over the rows in our `DataFrame`, it is noteworthy to understand that the Pandas actually keeps track on the `index` value as well. Hence, the contents of a single row actually contains not only the values, but also the `index` of that row (each row is a pandas Series!). 

- Let's see how `iterrows()` works by printing out the `TEMP` value on each row using a `for`-loop:


In [17]:
# Iterate over the rows
for idx, row in data.iterrows():
    # Print the index value
    print('Index:', idx)
    
    # Print the row
    print('Temp F:', row["TEMP_F"], "\n")
    
    break

Index: 0
Temp F: 6.0 



<div class="alert alert-info">

**break**

When developing a for-loop, you don't always need to go trough the whole loop if you just want to test things out. 
[break](https://www.tutorialspoint.com/python/python_break_statement.htm) statement in Python terminates the current loop after the first iteration and we used it here just to test check out the values on the first row.
    
</div>


We can see that the `idx` variable indeed contains the index value at position 0 (the first row) and the `row` variable contains all the data from that given row stored as a pandas `Series`.

- Let's now create an empty column `TEMP_C` for the Celsius temperatures and update the values into that column using the `fahr_to_celsius` function we defined earlier:


In [18]:
# Create an empty column for the DataFrame where the values will be stored
new_column = "TEMP_C"
data[new_column] = None

# Iterate over the rows 
for idx, row in data.iterrows():
    # Convert the Fahrenheit to Celsius
    celsius = fahr_to_celsius(row['TEMP_F'])
    
    # Update the value of 'Celsius' column with the converted value
    data.at[idx, new_column] = celsius

<div class="alert alert-info">

**.at or .loc?**

Here, you could also use `data.loc[idx, new_column] = celsius` to achieve the same result. 
    
If you only need to access a single value in a DataFrame, [DataFrame.at](https://pandas.pydata.org/pandas-docs/version/0.25/reference/api/pandas.DataFrame.at.html) is faster compared to [DataFrame.loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) which is designed for accessing groups of rows and columns.
    
</div>

- Let's see what we have now.

In [19]:
data.head(10)

STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN   TEMP_C
0           29350  191701010600  320.0    5.0   NaN     6.0  NaN  NaN -14.4444
1           29350  191701011300  320.0    7.0   NaN     2.0  NaN  NaN -16.6667
2           29350  191701012000  320.0    5.0   NaN    -8.0  NaN  NaN -22.2222
3           29350  191701021300    NaN    0.0   NaN    -4.0  NaN  NaN      -20
4           29350  191701022000    NaN    0.0   NaN   -12.0  NaN  NaN -24.4444
5           29350  191701030600    NaN    0.0   NaN    -8.0  NaN  NaN -22.2222
6           29350  191701031300    NaN    0.0   NaN     0.0  NaN  NaN -17.7778
7           29350  191701032000  320.0    5.0   NaN     2.0  NaN  NaN -16.6667
8           29350  191701040600  110.0   14.0   NaN     5.0  NaN  NaN      -15
9           29350  191701041300   70.0    2.0   NaN     6.0  NaN  NaN -14.4444

Great! Now we have converted our temperatures into Celsius by using our self-made function.

### Applying a function

Pandas DataFrames and Series also have a dedicated method `.apply()` for applying functions on columns (or rows!). When using `.apply()`, we pass the function name (without parenthesis!) as an argument:

In [20]:
data["TEMP_C"] = data["TEMP_F"].apply(fahr_to_celsius)
data.head()

STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29350  191701010600  320.0    5.0   NaN     6.0  NaN  NaN   
1           29350  191701011300  320.0    7.0   NaN     2.0  NaN  NaN   
2           29350  191701012000  320.0    5.0   NaN    -8.0  NaN  NaN   
3           29350  191701021300    NaN    0.0   NaN    -4.0  NaN  NaN   
4           29350  191701022000    NaN    0.0   NaN   -12.0  NaN  NaN   

      TEMP_C  
0 -14.444444  
1 -16.666667  
2 -22.222222  
3 -20.000000  
4 -24.444444

**Note:** pay attention which column you are applying the function on! Running this code: `data.apply(fahr_to_celsius)` would not give an error, but the results also don't make much sense.

<div class="alert alert-info">

**Should I use .iterrows() or .apply()?**

We are teaching the `.iterrows()` method because it helps to understand the structure of a DataFrame and the process of looping trough DataFrame rows. However, using `.apply()` is often more efficient in terms of execution time. 
    
    
At this point, the most important thing is that you understand what happens when you are modifying the values in a pandas DataFrame. When doing the course exercises, either of these approaches is ok!
    
</div>

## Parsing dates

We will eventually want to group our data based on month in order to see if April temperatures in 2019 were higher than average. Currently, the date and time information is stored in the column `TIME`:

`YR--MODAHRMN = YEAR-MONTH-DAY-HOUR-MINUTE IN GREENWICH MEAN TIME (GMT)`

Let's have a closer look at the date and time information we have by checking the values in that column, and their data type:

In [21]:
data["TIME"].head(10)

0    191701010600
1    191701011300
2    191701012000
3    191701021300
4    191701022000
5    191701030600
6    191701031300
7    191701032000
8    191701040600
9    191701041300
Name: TIME, dtype: int64

In [22]:
data["TIME"].tail(10)

559657    201910012050
559658    201910012100
559659    201910012120
559660    201910012150
559661    201910012200
559662    201910012220
559663    201910012250
559664    201910012300
559665    201910012320
559666    201910012350
Name: TIME, dtype: int64

The `TIME` column contains several observations per day (and even several observations per hour). The timestamp for the first observation is `191701010600`, i.e. from 1st of January 1917, and the timestamp for the latest observation is `201910012350` (from last week, by the time of writing this!)

In [23]:
data["TIME"].dtypes

dtype('int64')

And the information is stored as integer values.

There are several different options for proceeding from here. The bottom line is, that we would want to **aggregate the data on a monthly level**, and in order to do so we need to "label" each row of data based on the month when the record was observed. 
In practice, we could create a new column (or an index), which contains information about the month (including the year, but excluding hours and minutes).

### String slicing

One approach would be to convert the date and time information into character strings and "cut" the needed information from the [string objects](https://docs.python.org/3/tutorial/introduction.html#strings). If we look at the latest time stamp in the data (`201910012350`), you can see that there is a systematic pattern `YEAR-MONTH-DAY-HOUR-MINUTE`. Four first characters represent the year, and six first characters are year + month!

In [24]:
date = "201910012350"
date[0:6]

'201910'

Doing this in pandas requires two steps:
  1. Convert the `TIME` column from `int` into `str` datatype.
  2. Slice the correct range of characters from the character string using [pandas.Series.str.slice()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.slice.html)

- Let's convert the time into string. And check that the data type changes:

In [25]:
# Convert to string
data['TIME_STR'] = data['TIME'].astype(str)

In [26]:
# SLice the string
data['MONTH_STR'] = data['TIME_STR'].str.slice(start=0, stop=6)

# Let's see what we have
print(data.head())

   STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29350  191701010600  320.0    5.0   NaN     6.0  NaN  NaN   
1           29350  191701011300  320.0    7.0   NaN     2.0  NaN  NaN   
2           29350  191701012000  320.0    5.0   NaN    -8.0  NaN  NaN   
3           29350  191701021300    NaN    0.0   NaN    -4.0  NaN  NaN   
4           29350  191701022000    NaN    0.0   NaN   -12.0  NaN  NaN   

      TEMP_C      TIME_STR MONTH_STR  
0 -14.444444  191701010600    191701  
1 -16.666667  191701011300    191701  
2 -22.222222  191701012000    191701  
3 -20.000000  191701021300    191701  
4 -24.444444  191701022000    191701  


Nice! Now we have "labeled" the rows based on information about day of the year and hour of the day. However, let's have a look at a more clever way of dealing with dates and times..

### datetime

<div class="alert alert-info">

**Pandas datetime**

In pandas, we can convert dates and times into a new data type [datetime](https://docs.python.org/3.7/library/datetime.html) using [pandas.to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) function. First, it is important to understand the structure of the input data in order to avoid erroneous conversions, and that's why we first learned string slicing before introducing the datetime functionalities. 
    
Here is one example of how to convert the `TIME_STR`-column in our data set to datetime:
    
```
# Convert to datetime
data["DATE"] = pd.to_datetime(data["TIME_STR"])

```
   
        
If needed, you can use the `format` parameter to define the output datetime format according to [strftime(format) method](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior). together with `exact=False`, for example like this: 
    
```
# Convert to datetime
data["DATE"] = pd.to_datetime(data["TIME_STR"], format='%Y%m%d%H', exact=False)

```
In this example, `exact=False` drops out minutes and secods, because they are not included in the specified formatting.
 
</div>




In [27]:
# Convert to datetime
data["DATE"] = pd.to_datetime(data["TIME_STR"])
data["DATE"].head()

0   1917-01-01 06:00:00
1   1917-01-01 13:00:00
2   1917-01-01 20:00:00
3   1917-01-02 13:00:00
4   1917-01-02 20:00:00
Name: DATE, dtype: datetime64[ns]

Note: in this case, the data type of the values is `datetime`. 

<div class="alert alert-info">

**Pandas Series datetime properties**
    
There are several methods available for accessing information about the properties of datetime values. Read more from the pandas documentation about [datetime properties](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#datetime-properties).
</div>

Now, we can extract different time units based on the datetime-column using the [pandas.Series.dt accessor](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html):

In [28]:
data['DATE'].dt.year

0         1917
1         1917
2         1917
3         1917
4         1917
5         1917
6         1917
7         1917
8         1917
9         1917
10        1917
11        1917
12        1917
13        1917
14        1917
15        1917
16        1917
17        1917
18        1917
19        1917
20        1917
21        1917
22        1917
23        1917
24        1917
25        1917
26        1917
27        1917
28        1917
29        1917
          ... 
559637    2019
559638    2019
559639    2019
559640    2019
559641    2019
559642    2019
559643    2019
559644    2019
559645    2019
559646    2019
559647    2019
559648    2019
559649    2019
559650    2019
559651    2019
559652    2019
559653    2019
559654    2019
559655    2019
559656    2019
559657    2019
559658    2019
559659    2019
559660    2019
559661    2019
559662    2019
559663    2019
559664    2019
559665    2019
559666    2019
Name: DATE, Length: 559667, dtype: int64

We can also combine the datetime functionalities with other methods from pandas. For example, we can check the number of unique years in our input data: 

In [29]:
data['DATE'].dt.year.nunique()

69

<div class="alert alert-info">

**TASK:**

- Create two new columns: `YEAR` and `MONTH` based on the date column
</div>

In [30]:
data['YEAR'] = data['DATE'].dt.year
data['MONTH'] = data['DATE'].dt.month

data.head()

STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29350  191701010600  320.0    5.0   NaN     6.0  NaN  NaN   
1           29350  191701011300  320.0    7.0   NaN     2.0  NaN  NaN   
2           29350  191701012000  320.0    5.0   NaN    -8.0  NaN  NaN   
3           29350  191701021300    NaN    0.0   NaN    -4.0  NaN  NaN   
4           29350  191701022000    NaN    0.0   NaN   -12.0  NaN  NaN   

      TEMP_C      TIME_STR MONTH_STR                DATE  YEAR  MONTH  
0 -14.444444  191701010600    191701 1917-01-01 06:00:00  1917      1  
1 -16.666667  191701011300    191701 1917-01-01 13:00:00  1917      1  
2 -22.222222  191701012000    191701 1917-01-01 20:00:00  1917      1  
3 -20.000000  191701021300    191701 1917-01-02 13:00:00  1917      1  
4 -24.444444  191701022000    191701 1917-01-02 20:00:00  1917      1

## Aggregating data in Pandas by grouping

Here, we will learn how to use [pandas.DataFrame.groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) which is a handy method for compressing large amounts of data and computing statistics for subgroups.

Our practical task is to calculate the average temperatures for each month

This can be done by aggregating the data, i.e.:

  1. **grouping the data** based on year and month
  2. Calculating the average for each month (each group) either by using a for-loop or directly from the grouped object
  3. Storing those values into **a new DataFrame** `monthly_data`

Before we start grouping the data, let's once more check how our input data looks like:

In [31]:
print("number of rows:", len(data))
data.head()

number of rows: 559667


STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29350  191701010600  320.0    5.0   NaN     6.0  NaN  NaN   
1           29350  191701011300  320.0    7.0   NaN     2.0  NaN  NaN   
2           29350  191701012000  320.0    5.0   NaN    -8.0  NaN  NaN   
3           29350  191701021300    NaN    0.0   NaN    -4.0  NaN  NaN   
4           29350  191701022000    NaN    0.0   NaN   -12.0  NaN  NaN   

      TEMP_C      TIME_STR MONTH_STR                DATE  YEAR  MONTH  
0 -14.444444  191701010600    191701 1917-01-01 06:00:00  1917      1  
1 -16.666667  191701011300    191701 1917-01-01 13:00:00  1917      1  
2 -22.222222  191701012000    191701 1917-01-01 20:00:00  1917      1  
3 -20.000000  191701021300    191701 1917-01-02 13:00:00  1917      1  
4 -24.444444  191701022000    191701 1917-01-02 20:00:00  1917      1

We have quite a few rows of weather data, and several observations per day (and even per hour). **Our goal is to create an aggreated data frame that would have only one row per month!**

Let's **group** our data based on unique year and month combinations

In [32]:
grouped = data.groupby(["YEAR", "MONTH"])

<div class="alert alert-info">

**NOTE:**
    
Here you could also group the data based on the `MONTH_STR` column to achieve the same result:
    
```
# Group the data 
grouped = data.groupby('MONTH_STR')
```
</div>

- Let's see what we have now.

In [33]:
# What is the type?
print("Type:\n", type(grouped))

# How many?
print("Length:\n", len(grouped))

Type:
 <class 'pandas.core.groupby.groupby.DataFrameGroupBy'>
Length:
 826


Okey, interesting. Now we have a new object with type `DataFrameGroupBy`.

<div class="alert alert-info">

**TASK:**

Think: what does the number of groups (length of the grouped object) tell us?
</div>

In [34]:
# Answer: the length of the grouped object should be the same as
data["MONTH_STR"].nunique()

# in other words, the number of groups is the number of unique year and month combinations in our data

826

There are several methods we can use for extracting information from the grouped data. See [documentation for Pandas GroupBy objects](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html) for a comprehensive overview. 

**Checking group names:**

In [35]:
# Check the "names" of each group (comment out the next row if you want to print out the result!)
#grouped.groups.keys()

**Accessing data for one group:**

- Let's check the contents for a group representing August 2019 (name of that group is `(2019, 4)` if you grouped the data based on datetime columns `YEAR` and `MONTH`). We can get the values of that hour from the grouped object using the `get_group()` -method:

In [36]:
# Specify the time of the first hour (as text)
month = (2019, 4)

# Select the group
group1 = grouped.get_group(month)

# Let's see what we have
print(group1)

        STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F   MAX   MIN  \
546832           29350  201904010000  290.0    9.0   NaN    29.0   NaN   NaN   
546833           29350  201904010050  330.0    6.0   NaN    28.0   NaN   NaN   
546834           29350  201904010100  280.0    9.0   NaN    29.0   NaN   NaN   
546835           29350  201904010120  310.0    7.0   NaN    28.0   NaN   NaN   
546836           29350  201904010150  320.0    3.0   NaN    27.0   NaN   NaN   
546837           29350  201904010200  290.0    2.0   NaN    28.0   NaN   NaN   
546838           29350  201904010220  290.0    5.0   NaN    27.0   NaN   NaN   
546839           29350  201904010250  300.0    6.0   NaN    27.0   NaN   NaN   
546840           29350  201904010300  300.0    9.0   NaN    28.0   NaN   NaN   
546841           29350  201904010320  320.0    8.0   NaN    27.0   NaN   NaN   
546842           29350  201904010350  310.0   10.0   NaN    27.0   NaN   NaN   
546843           29350  201904010400  30

Ahaa! As we can see, a single group contains a **DataFrame** with values only for that specific month. Let's check the DataType of this group:

In [37]:
type(group1)

pandas.core.frame.DataFrame

So, one group is a pandas DataFrame! This is really useful, because we can now use all the familiar DataFrame methods for calculating statistics etc for this spesific group. 
We can, for example, calculate the average values for all variables using the statistical functions that we have seen already (e.g. mean, std, min, max, median, etc.).

We can do that by using the `mean()` -function that we already used during the Lesson 5. 

- Let's calculate the mean for following attributes all at once:
   - `DIR`, 
   - `SPEED`, 
   - `GUST`, 
   - `TEMP`, 
   - `TEMP_C`
   - `MONTH` 

In [38]:
# Specify the columns that will be part of the calculation
mean_cols = ['DIR', 'SPEED', 'GUST', 'TEMP_F', 'TEMP_C', 'MONTH']

# Calculate the mean values all at one go
mean_values = group1[mean_cols].mean()

# Let's see what we have
print(mean_values)

DIR       399.197218
SPEED       6.020893
GUST       21.416667
TEMP_F     40.585002
TEMP_C      4.769446
MONTH       4.000000
dtype: float64


Nice, now we have averaged our data and e.g. the mean Celsius temperature seems to be about right when comparing to the original values above. 

Here we saw how you can access data from a single group. For getting information about all groups (all months)we can a `for` -loop or methods available in the grouped object.

**For-loops and grouped objects:**

When iterating over the groups in our `DataFrameGroupBy` -object
it is important to understand that a single group in our `DataFrameGroupBy` actually contains not only the actual values, but also information about the `key` that was used to do the grouping. Hence, when iterating over the data we need to assign the `key` and the values into separate variables.

- Let's see how we can iterate over the groups and print the key and the data from a single group (again using `break` to only see what is happening).


In [39]:
# Iterate over groups
for key, group in grouped:
    # Print key and group
    print("Key:\n", key)
    print("\nFirst rows of data in this group:\n", group.head())
    
    # Stop iteration with break command
    break

Key:
 (1917, 1)

First rows of data in this group:
    STATION_NUMBER          TIME    DIR  SPEED  GUST  TEMP_F  MAX  MIN  \
0           29350  191701010600  320.0    5.0   NaN     6.0  NaN  NaN   
1           29350  191701011300  320.0    7.0   NaN     2.0  NaN  NaN   
2           29350  191701012000  320.0    5.0   NaN    -8.0  NaN  NaN   
3           29350  191701021300    NaN    0.0   NaN    -4.0  NaN  NaN   
4           29350  191701022000    NaN    0.0   NaN   -12.0  NaN  NaN   

      TEMP_C      TIME_STR MONTH_STR                DATE  YEAR  MONTH  
0 -14.444444  191701010600    191701 1917-01-01 06:00:00  1917      1  
1 -16.666667  191701011300    191701 1917-01-01 13:00:00  1917      1  
2 -22.222222  191701012000    191701 1917-01-01 20:00:00  1917      1  
3 -20.000000  191701021300    191701 1917-01-02 13:00:00  1917      1  
4 -24.444444  191701022000    191701 1917-01-02 20:00:00  1917      1  


Okey so from here we can see that the `key` contains the name of the group (year, month).

- Let's see how we can create a DataFrame where we calculate the mean values for all those weather attributes that we were interested in. I will repeate slightly the earlier steps so that you can see and better understand what is happening.


In [40]:
# Create an empty DataFrame for the aggregated values
hourly_data = pd.DataFrame()

# The columns that we want to aggregate
mean_cols = ['DIR', 'SPEED', 'GUST', 'TEMP_F', 'TEMP_C', "MONTH"]

# Iterate over the groups
for key, group in grouped:
   # Aggregate the data
   mean_values = group[mean_cols].mean()

   # Add the ´key´ (i.e. the date+time information) into the aggregated values
   mean_values['YEAR_MONTH'] = key

   # Append the aggregated values into the DataFrame
   hourly_data = hourly_data.append(mean_values, ignore_index=True)

- Let's see what we have now:

In [41]:
print(hourly_data)

            DIR       GUST  MONTH      SPEED     TEMP_C     TEMP_F  YEAR_MONTH
0    226.250000        NaN    1.0   7.184783 -10.996377  12.206522   (1917, 1)
1    215.945946        NaN    2.0  11.738095 -14.212963   6.416667   (1917, 2)
2    185.769231        NaN    3.0   9.032258 -10.890084  12.397849   (1917, 3)
3    173.717949        NaN    4.0   8.611111   0.746914  33.344444   (1917, 4)
4    261.851852        NaN    5.0  11.591398   6.911589  44.440860   (1917, 5)
5    211.315789        NaN    6.0   7.777778  17.820988  64.077778   (1917, 6)
6    225.487805        NaN    7.0   6.677419  16.612903  61.903226   (1917, 7)
7    161.971831        NaN    8.0   6.150538  18.500597  65.301075   (1917, 8)
8    237.027027        NaN    9.0   9.966667   9.098765  48.377778   (1917, 9)
9    162.619048        NaN   10.0  12.784946   6.164875  43.096774  (1917, 10)
10   199.529412        NaN   11.0  13.322222  -0.364198  31.344444  (1917, 11)
11   186.956522        NaN   12.0   8.096774  -5.746

Awesome! Now we have aggregated our data based on daily averages and we have a new DataFrame called `hourly_data` where all those aggregated values are stored.

**Mean for all groups at once**

We can also achieve the same result by computing the mean of all columns for all groups in the grouped object:

In [42]:
grouped.mean()

STATION_NUMBER          TIME         DIR      SPEED       GUST  \
YEAR MONTH                                                                   
1917 1             29350.0  1.917012e+11  226.250000   7.184783        NaN   
     2             29350.0  1.917021e+11  215.945946  11.738095        NaN   
     3             29350.0  1.917032e+11  185.769231   9.032258        NaN   
     4             29350.0  1.917042e+11  173.717949   8.611111        NaN   
     5             29350.0  1.917052e+11  261.851852  11.591398        NaN   
     6             29350.0  1.917062e+11  211.315789   7.777778        NaN   
     7             29350.0  1.917072e+11  225.487805   6.677419        NaN   
     8             29350.0  1.917082e+11  161.971831   6.150538        NaN   
     9             29350.0  1.917092e+11  237.027027   9.966667        NaN   
     10            29350.0  1.917102e+11  162.619048  12.784946        NaN   
     11            29350.0  1.917112e+11  199.529412  13.322222        NaN   
     12            29350.0  1.917122e+11  186.956522   8.096774        NaN   
1918 1             29350.0  1.918012e+11  213.924051  11.580645        NaN   
     2             29350.0  1.918021e+11  201.285714   8.011905        NaN   
     3             29350.0  1.918032e+11  229.714286   8.838710        NaN   
     4             29350.0  1.918042e+11  171.692308   6.244444        NaN   
     5             29350.0  1.918052e+11  213.294118   9.913978        NaN   
     6             29350.0  1.918062e+11  211.666667  10.800000        NaN   
     7             29350.0  1.918072e+11  195.810811   7.860215        NaN   
     8             29350.0  1.918082e+11  205.909091   6.182796        NaN   
     9             29350.0  1.918092e+11  190.142857   9.444444        NaN   
     10            29350.0  1.918102e+11  185.925926   9.408602        NaN   
     11            29350.0  1.918112e+11  189.610390  10.363636        NaN   
     12            29350.0  1.918122e+11  149.746835  10.333333        NaN   
1919 1             29350.0  1.919012e+11  129.589041   6.526882        NaN   
     2             29350.0  1.919021e+11  241.551724   7.702381        NaN   
     3             29350.0  1.919032e+11  202.750000  12.688172        NaN   
     4             29350.0  1.919042e+11  198.030303   9.149425        NaN   
     5             29350.0  1.919052e+11  211.617647   7.827957        NaN   
     6             29350.0  1.919062e+11  189.846154   8.275862        NaN   
...                    ...           ...         ...        ...        ...   
2017 5             29350.0  2.017052e+11  405.082453   7.452416  17.250000   
     6             29350.0  2.017062e+11  406.696117   7.273412  18.533613   
     7             29350.0  2.017072e+11  395.458746   5.807256  14.882353   
     8             29350.0  2.017082e+11  425.223558   5.856287  15.767196   
     9             29350.0  2.017092e+11  346.161481   5.072941  15.480000   
     10            29350.0  2.017102e+11  305.445283   6.762276  16.729469   
     11            29350.0  2.017112e+11  363.135977   6.149673  15.989189   
     12            29350.0  2.017122e+11  353.396004   7.093086  16.781116   
2018 1             29350.0  2.018012e+11  343.304731   6.006312  16.835294   
     2             29350.0  2.018021e+11  278.572392   6.137827  14.370968   
     3             29350.0  2.018032e+11  357.101570   6.394347  25.312500   
     4             29350.0  2.018042e+11  375.255561   5.652785  23.125000   
     5             29350.0  2.018052e+11  421.332520   6.625115  24.476190   
     6             29350.0  2.018062e+11  433.026392   8.346118  27.115942   
     7             29350.0  2.018072e+11  363.347531   6.089236  20.000000   
     8             29350.0  2.018082e+11  473.589453   5.978913  24.724138   
     9             29350.0  2.018092e+11  445.613356   6.511075  23.695238   
     10            29350.0  2.018102e+11  405.446437   6.158283  18.812500   
     11            29350.0  2.0181

## Detecting warm months

Now, we have aggregated our data on monthly level and all we need to do is to check which years had the warmest April temperatures. A simple approach is to select all aprils from the data, group the data and check which group(s) have the highest mean value:

- select all records that are from April (regardless of the year):

In [43]:
aprils = data[data["MONTH"]==4]

- take a subset of columns that might contain interesting information:

In [45]:
aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C','YEAR', 'MONTH']]

- group by year and month:

In [46]:
grouped = aprils.groupby(by=["YEAR", "MONTH"])

- calculate mean for each group:

In [51]:
monthly_mean = grouped.mean()
monthly_mean.head()

STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
1917 4               29350  33.344444  0.746914
1918 4               29350  38.811111  3.783951
1919 4               29350  33.988372  1.104651
1920 4               29350  39.277778  4.043210
1921 4               29350  45.144444  7.302469

- check the highest temperature values (sort the data frame in a descending order):

In [52]:
monthly_mean.sort_values(by="TEMP_C", ascending=False).head(10)

STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
1921 4               29350  45.144444  7.302469
1925 4               29350  40.777778  4.876543
2011 4               29350  40.670108  4.816727
2019 4               29350  40.585002  4.769446
2001 4               29350  39.662827  4.257126
1920 4               29350  39.277778  4.043210
1990 4               29350  39.075949  3.931083
1999 4               29350  38.983356  3.879642
1918 4               29350  38.811111  3.783951
2007 4               29350  38.713256  3.729587

How did April 2019 rank at this observation station 🌡️? 

## Repeating the data analysis with larger dataset


Finally, let's repeat the data analysis steps above for all the available data we have (!!). First, confirm the path to the **folder** where all the input data are located. 
The idea is, that we will repeat the analysis process for each input file using a (rather long) for-loop! Here are all the main alaysis steps with some additional print out info - all in one long code cell:

In [53]:
# Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
data = pd.read_csv(fp, delim_whitespace=True, usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], na_values=['*', '**', '***', '****', '*****', '******'])

# Rename the columns
new_names = {'USAF':'STATION_NUMBER','YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST', 'TEMP':'TEMP_F'}
data = data.rename(columns=new_names)

#Print info about the current input file:
print("STATION NUMBER:", data.at[0,"STATION_NUMBER"])
print("NUMBER OF OBSERVATIONS:", len(data))

# Create column
col_name = 'TEMP_C'
data[col_name] = None

# Convert tempetarues from Fahrenheits to Celsius
data["TEMP_C"] = data["TEMP_F"].apply(fahr_to_celsius)

# Convert TIME to string 
data['TIME_STR'] = data['TIME'].astype(str)

# Parse year and month
data['MONTH'] = data['TIME_STR'].str.slice(start=5, stop=6).astype(int)
data['YEAR'] = data['TIME_STR'].str.slice(start=0, stop=4).astype(int)

# Extract observations for the months of April 
aprils = data[data['MONTH']==4]

# Take a subset of columns
aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C', 'YEAR', 'MONTH']]

# Group by year and month
grouped = aprils.groupby(by=["YEAR", "MONTH"])

# Get mean values for each group
monthly_mean = grouped.mean()

# Print info
print(monthly_mean.sort_values(by="TEMP_C", ascending=False).head(5))
print("\n")

STATION NUMBER: 29350
NUMBER OF OBSERVATIONS: 559667
            STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
1921 4               29350  45.144444  7.302469
1925 4               29350  40.777778  4.876543
2011 4               29350  40.670108  4.816727
2019 4               29350  40.585002  4.769446
2001 4               29350  39.662827  4.257126




We will use `glob()` -function from module `glob` to list our input files. 

In [54]:
import glob

In [55]:
file_list = glob.glob(r'C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\*txt')

In [56]:
print("Number of files in the list", len(file_list))
print(file_list)

Number of files in the list 15
['C:\\LocalData\\VUOKKHEI\\codes\\Geo-Python\\data\\weather_data\\028360.txt', 'C:\\LocalData\\VUOKKHEI\\codes\\Geo-Python\\data\\weather_data\\028690.txt', 'C:\\LocalData\\VUOKKHEI\\codes\\Geo-Python\\data\\weather_data\\028750.txt', 'C:\\LocalData\\VUOKKHEI\\codes\\Geo-Python\\data\\weather_data\\028970.txt', 'C:\\LocalData\\VUOKKHEI\\codes\\Geo-Python\\data\\weather_data\\029070.txt', 'C:\\LocalData\\VUOKKHEI\\codes\\Geo-Python\\data\\weather_data\\029110.txt', 'C:\\LocalData\\VUOKKHEI\\codes\\Geo-Python\\data\\weather_data\\029170.txt', 'C:\\LocalData\\VUOKKHEI\\codes\\Geo-Python\\data\\weather_data\\029350.txt', 'C:\\LocalData\\VUOKKHEI\\codes\\Geo-Python\\data\\weather_data\\029440.txt', 'C:\\LocalData\\VUOKKHEI\\codes\\Geo-Python\\data\\weather_data\\029500.txt', 'C:\\LocalData\\VUOKKHEI\\codes\\Geo-Python\\data\\weather_data\\029700.txt', 'C:\\LocalData\\VUOKKHEI\\codes\\Geo-Python\\data\\weather_data\\029720.txt', 'C:\\LocalData\\VUOKKHEI\\codes\

Now, you should have all the relevant file names in a list, and we can loop over the list using a for-loop:

In [57]:
for fp in file_list:
    print(fp)

C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\028360.txt
C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\028690.txt
C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\028750.txt
C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\028970.txt
C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\029070.txt
C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\029110.txt
C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\029170.txt
C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\029350.txt
C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\029440.txt
C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\029500.txt
C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\029700.txt
C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\029720.txt
C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\029740.txt
C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weather_data\029810.txt
C:\LocalData\VUOKKHEI\codes\Geo-Python\data\weat

In [58]:
# Repeat the analysis steps for each input file:
for fp in file_list:

    # Read selected columns of  data using varying amount of spaces as separator and specifying * characters as NoData values
    data = pd.read_csv(fp, delim_whitespace=True, usecols=['USAF','YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'], na_values=['*', '**', '***', '****', '*****', '******'])

    # Rename the columns
    new_names = {'USAF':'STATION_NUMBER','YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST', 'TEMP':'TEMP_F'}
    data = data.rename(columns=new_names)

    #Print info about the current input file:
    print("STATION NUMBER:", data.at[0,"STATION_NUMBER"])
    print("NUMBER OF OBSERVATIONS:", len(data))

    # Create column
    col_name = 'TEMP_C'
    data[col_name] = None

    # Convert tempetarues from Fahrenheits to Celsius
    data["TEMP_C"] = data["TEMP_F"].apply(fahr_to_celsius)

    # Convert TIME to string 
    data['TIME_STR'] = data['TIME'].astype(str)

    # Parse year and month
    data['MONTH'] = data['TIME_STR'].str.slice(start=5, stop=6).astype(int)
    data['YEAR'] = data['TIME_STR'].str.slice(start=0, stop=4).astype(int)

    # Extract observations for the months of April 
    aprils = data[data['MONTH']==4]

    # Take a subset of columns
    aprils = aprils[['STATION_NUMBER','TEMP_F', 'TEMP_C', 'YEAR', 'MONTH']]

    # Group by year and month
    grouped = aprils.groupby(by=["YEAR", "MONTH"])

    # Get mean values for each group
    monthly_mean = grouped.mean()

    # Print info
    print(monthly_mean.sort_values(by="TEMP_C", ascending=False).head(5))
    print("\n")

STATION NUMBER: 28360
NUMBER OF OBSERVATIONS: 193825
            STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
1937 4               28360  38.738095  3.743386
2011 4               28360  36.699571  2.610873
1921 4               28360  36.622222  2.567901
2002 4               28360  36.500000  2.500000
2019 4               28360  34.979138  1.655076


STATION NUMBER: 28690
NUMBER OF OBSERVATIONS: 542788
            STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
2011 4               28690  35.430640  1.905911
2019 4               28690  35.215114  1.786174
2016 4               28690  35.031103  1.683946
1989 4               28690  34.612766  1.451537
2002 4               28690  34.279855  1.266586


STATION NUMBER: 28750
NUMBER OF OBSERVATIONS: 474562
            STATION_NUMBER     TEMP_F    TEMP_C
YEAR MONTH                                     
1989 4               28750  39.008403  3.893557
1983 4               

How about now, how did April 2019 rank across different stations?